In [1]:
# 1. Imports
import csv

In [2]:
# 2. Classes

# Account class
class Account:
    """
    Base account class used by Checking, Savings, and Credit.

    Attributes:
    account_id: four-digit account ID as string
    balance: current balance of the account (for credit accounts shows amount owed) as float
    interest: rate applies to account as float

    """

    # Fixed _init__ misspelling
    def __init__(self, account_id="0000", balance=0.0, interest=0):
        """
        Initialize a general account
        """
        self.account_id = account_id
        self.balance = balance
        self.interest = interest

    def __str__(self):
        """
        Print account data
        """
        return f"ID: {self.account_id}, Balance: ${self.balance:.2f}, Interest: {self.interest*100}%"  
        
    # Get and Set methods for account ID
    def get_account_id(self):
        """
        Return the account ID
        """
        # print("getting account ID")
        return self._account_id
    
    def set_account_id(self, value):
        """
        Set the account ID
        """
        # print("setting account id to", value)

        if not isinstance(value, str) or len(value) != 4:
            raise ValueError("Account ID must be a 4-digit string")
        
        for char in value:
            if char not in "0123456789":
                raise ValueError("Account ID must contain numeric digits")
        
        self._account_id = value

    account_id = property(get_account_id, set_account_id)

    # Get and Set methods for balance
    def get_balance(self):
        """
        Return the current balance
        """
        # print("getting balance")
        return self._balance
    
    def set_balance(self, value):
        """
        Set the account balance
        """
        # print("setting balance to", value)

        if not isinstance(value, (int, float)):
            raise TypeError("Balance must be a number")
        
        self._balance = float(value)

    balance = property(get_balance, set_balance)

    # Core methods
    def deposit(self, amount):
        """
        Add money to the account
        """
        self.balance += amount

    def withdraw(self, amount):
        """
        Remove money from the account if there is enough funds
        """
        if amount > self.balance:
            print("Withdrawal denied: insufficient funds")
            return False
        self.balance -= amount
        return True


# Checking Classes
class Checking(Account):
    """
    Checking account class
    Interest rate is always 0%
    """

    def __init__(self, account_id, balance):
        super().__init__(account_id, balance, interest=0)


# Savings Account Class
class Savings(Account):
    """
    Savings account class
    Interest rate is always 1%
    """

    def __init__(self, account_id, balance):
        super().__init__(account_id, balance, interest=0.01)


# Credit Account
class Credit(Account):
    """
    Credit account class. 

    Attributes:
    credit_limit: Maximum amount allowed to be charged

    """

    def __init__(self, account_id, balance, credit_limit):
        super().__init__(account_id, balance, interest=0.30)
        self.credit_limit = credit_limit

    # Get and Set methods for credit limit
    def get_credit_limit(self):
        """
        Return credit limit
        """
        # print("Getting credit limit.")
        return self._credit_limit
    
    def set_credit_limit(self, value):
        """
        Set the credit limit to value
        """
        # print("Setting credit limit to", value)

        if not isinstance(value, (int, float)):
            raise TypeError("Credit limit must be a number")
        
        self._credit_limit = float(value)

    credit_limit = property(get_credit_limit, set_credit_limit)

    # Credit Methods
    def charge(self, amount):
        """
        Add a charge to the credit balance if it
        does not exceed the credit limit.
        """
        if self.balance + amount > self.credit_limit:
            print("Charge denied: credit limit exceeded")
            return False
        
        self.balance += amount
        return True
    
    def make_payment(self, amount):
        """
        Reduce the credit balance
        """
        if amount > self.balance:
            print("Payment denied: too large")
            return False
        
        self.balance -= amount
        return True


# Customer Class
class Customer:
    """
    Represent one customer with three accounts.

    Attributes;
    username: string representing a person uniquely
    checking: Checking object
    Savings: Savings object
    Credit: Credit object
    """

    def __init__(self, username, checking, savings, credit):
        self.username = username
        self.checking = checking
        self.savings = savings
        self.credit = credit

    def __str__(self):
        """
        Print all customer info
        """
        return (
            f"\nCustomer: {self.username}\n"
            f"Checking: {self.checking}\n"
            f"Savings: {self.savings}\n"
            f"Credit: {self.credit}, Limit: ${self.credit.credit_limit:.2f}\n"
        )


In [3]:
# 3. Global variable(s)
customerList = []

In [4]:
# 4. Customer Lookup Function

def findCustomers():
    # Accessing global list
    global customerList

    # Asking for username input
    print(f"Enter customer username:", flush=True)
    input_username = input()
    
    # Checking if customer exists
    for customer_name in customerList:
        if customer_name.username == input_username:
            return customer_name
    
    # Error message
    print(f"Username DOES NOT Exist!", flush=True)
    print(flush=True) # Blank Line/empty space

    # Recursion
    return findCustomers()

In [ ]:
# 5. File I/O Functions

def importCustomers():
    """
    Imports customer data from a CSV file
    Creates account objects
    Stores them in global customerList.
    """
    global customerList

    # Asking for user input on file location
    print(f"Enter file path:", flush=True)
    file_path = input()

    try:
        with open(file_path, newline='') as csvfile:
            reader = csv.reader(csvfile)
            next(reader) # skipping header row

            customerList.clear()

            # Extracting csv row infor and creating objects
            for row in reader:
                username = row[0]

                checking = Checking(row[1], float(row[2]))
                savings = Savings(row[3], float(row[4]))
                credit = Credit(row[5], float(row[6]), float(row[7]))

                new_customer = Customer(username, checking, savings, credit)
                customerList.append(new_customer)
        
        # Success message
        print(f"File Successfully Imported", flush=True)
        print(flush=True)

    # Error message if importing failed
    except:
        print(f"Invalid File Address! Please try again", flush=True)
        print(flush=True)
        importCustomers()


def exportCustomers():
    """
    Export customer data to CSV file

    """
    global customerList

    # Asking for user input on file name for exporting
    print(f"Enter file name to export to:", flush=True)
    file_path = input()

    # Ensures file has .csv extension
    if not file_path.endswith(".csv"):
        file_path += ".csv"

    # Open new csv for writing
    try:
        with open(file_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)

            # Header row
            writer.writerow([
                "username","checking_id", "checking_balance", "savings_id", 
                "savings_balance", "credit_id", "credit_balance", "credit_limit"
            ])

            # Create customer info
            for account_holder in customerList:
                writer.writerow([
                    account_holder.username, account_holder.checking.account_id, 
                    account_holder.checking.balance, account_holder.savings.account_id,
                    account_holder.savings.balance, account_holder.credit.account_id,
                    account_holder.credit.balance, account_holder.credit.credit_limit
                ])
        
        # Success message
        print(f"File Successfully Exported", flush=True)
        print(flush=True)

    # Error message
    except:
        print(f"Error Exporting File! Please try again", flush=True)
        print(flush=True)
        exportCustomers()


In [ ]:
# 6. Banking Operations

def viewCustomers():
    # Accessing global list
    global customerList

    # Checking if customer list is empty
    if not customerList:
        print("No Existing Customers", flush=True)
        print(flush=True)
        return

    # Displaying info for each customer in list
    for customer in customerList:
        print(customer, flush=True)
        print(flush=True)


def deposit():
    # Finding customer object by username
    account_holder = findCustomers()

    # Asking which account to deposit into
    print("Select Account (Checkings/Savings):", flush=True)
    selected_account = input().lower()

    # Mapping user choice to correct account object
    if selected_account == "checkings":
        selected_account = account_holder.checking
    elif selected_account == "savings":
        selected_account = account_holder.savings
    else:
        print("Invalid Account Type!", flush=True)
        print(flush=True)
        return deposit()

    # Asking for deposit amount
    print("Enter Deposit Amount:", flush=True)
    try:
        transaction_amount = float(input())
        if transaction_amount <= 0:
            raise ValueError
    except:
        print("Invalid Amount!", flush=True)
        print(flush=True)
        return deposit()

    # Performing deposit using account method
    selected_account.deposit(transaction_amount)
    
    print("Deposit Successful", flush=True)
    print(flush=True)


def withdraw():
    # Finding customer object by username
    account_holder = findCustomers()

    # Asking which account to withdraw from
    print("Select Account (Checkings/Savings):", flush=True)
    selected_account = input().lower()

    # Mapping user choice to correct account object
    if selected_account == "checkings":
        selected_account = account_holder.checking
    elif selected_account == "savings":
        selected_account = account_holder.savings
    else:
        print("Invalid Account Type!", flush=True)
        print(flush=True)
        return withdraw()

    # Asking for withdrawal amount
    print("Enter Withdrawal Amount:", flush=True)
    try:
        transaction_amount = float(input())
        if transaction_amount <= 0:
            raise ValueError
    except:
        print("Invalid Amount!", flush=True)
        print(flush=True)
        return withdraw()

    # Attempting withdrawal using account method
    if selected_account.withdraw(transaction_amount):
        print("Withdrawal Successful", flush=True)
    else:
        print("Withdrawal Unsuccessful: Insufficient Funds", flush=True)

    print(flush=True)


def creditCharge():
    # Finding customer object by username
    account_holder = findCustomers()

    # Asking for credit charge amount
    print("Enter Charge Amount:", flush=True)
    try:
        transaction_amount = float(input())
        if transaction_amount <= 0:
            raise ValueError
    except:
        print("Invalid Amount!", flush=True)
        print(flush=True)
        return creditCharge()

    # Attempting credit charge using credit account method
    if account_holder.credit.charge(transaction_amount):
        print("Charge Successful", flush=True)
    else:
        print("Declined: Credit Limit Exceeded", flush=True)

    print(flush=True)


def creditPayment():
    # Finding customer object by username
    account_holder = findCustomers()

    # Selecting account to pay credit from
    print("Select Account (Checkings/Savings):", flush=True)
    selected_account = input().lower()

    # Mapping user choice to payment source account
    if selected_account == "checkings":
        payment_account = account_holder.checking
    elif selected_account == "savings":
        payment_account = account_holder.savings
    else:
        print("Invalid Account Type!", flush=True)
        print(flush=True)
        return creditPayment()

    # Asking for payment amount
    print("Enter payment amount:", flush=True)
    try:
        transaction_amount = float(input())
        if transaction_amount <= 0:
            raise ValueError
    except:
        print("Invalid Amount!", flush=True)
        print(flush=True)
        return creditPayment()

    # Checking if selected account has enough funds
    if transaction_amount > payment_account.balance:
        print("Payment Declined: Insufficient Funds", flush=True)
        print(flush=True)
        return

    # Attempting to apply payment to credit balance
    if not account_holder.credit.make_payment(transaction_amount):
        print("Payment Declined: Amount Exceeds Credit Balance", flush=True)
        print(flush=True)
        return

    # Withdrawing payment amount from selected account
    payment_account.withdraw(transaction_amount)

    print("Payment Successful", flush=True)
    print(flush=True)


In [ ]:
# 7. Interface

def interface():
    """
    Runs the menu for the banking system
    """

    # Importing customer data from CSV at program start
    importCustomers()

    # Main menu loop
    while True:

        # Displaying menu options
        print(
            f"\n----Bank Menu----\n"
            f"1. View Customers \n"
            f"2. Deposit \n"
            f"3. Withdraw \n"
            f"4. Credit Charge \n"
            f"5. Credit Payment \n"
            f"6. Exit", flush=True
        )

        # Collecting user menu selection
        print("Enter choice", flush=True)
        choice = input()

        # Viewing all customers
        if choice == "1":
            viewCustomers()

        # Depositing into checking or savings
        elif choice == "2":
            deposit()

        # Withdrawing from checking or savings
        elif choice == "3":
            withdraw()

        # Charging a credit account
        elif choice == "4":
            creditCharge()

        # Making a credit card payment
        elif choice == "5":
            creditPayment()

        # Exporting data and exiting program
        elif choice == "6":
            exportCustomers()
            print("Goodbye!")
            break

        # Handling invalid menu input
        else:
            print("Invalid Input")
            break


In [9]:
# 8. Calling Interface
interface()

Enter file path:
File Successfully Imported


----Bank Menu----
1. View Customers 
2. Deposit 
3. Withdraw 
4. Credit Charge 
5. Credit Payment 
6. Exit
Enter choice

Customer: amohr
Checking: ID: 1337, Balance: $43.00, Interest: 0%
Savings: ID: 0666, Balance: $101.45, Interest: 1.0%
Credit: ID: 1729, Balance: $5000.00, Interest: 30.0%, Limit: $5000.00



Customer: bbaggins
Checking: ID: 2890, Balance: $15345.49, Interest: 0%
Savings: ID: 2941, Balance: $15577483.00, Interest: 1.0%
Credit: ID: 3021, Balance: $0.00, Interest: 30.0%, Limit: $50000.00



Customer: emusk
Checking: ID: 0001, Balance: $21588737.58, Interest: 0%
Savings: ID: 0002, Balance: $1000000000.00, Interest: 1.0%
Credit: ID: 0003, Balance: $435678.58, Interest: 30.0%, Limit: $10000000.00



----Bank Menu----
1. View Customers 
2. Deposit 
3. Withdraw 
4. Credit Charge 
5. Credit Payment 
6. Exit
Enter choice
Enter customer username:
Select Account (Checkings/Savings):
Enter Deposit Amount:
Deposit Successful


----Bank